In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [16]:
!pip -q install configparser langchain google-generativeai chromadb > /dev/null
!pip -q install transformers huggingface_hub sentence_transformers > /dev/null

In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.environ['API_KEY'])

In [17]:
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.llms import GooglePalm

In [ ]:
#Then convert the transformers pipeline into fact_llm 

fact_llm = GooglePalm(temperature=0.1)

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt_open = PromptTemplate(template=template, input_variables=["question"])

open_chain = LLMChain(prompt=prompt_open,llm = fact_llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

print(open_chain.run(question))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 Justin Beiber was born in 1994, so the Super Bowl was in 2015. The team that won the Super Bowl in 2015 was the New England Patriots.


In [18]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
recurSplitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=20,
                                               length_function=len)

In [ ]:
with open('/content/linux_play.txt') as lin:
  txt_lin = lin.read()

linux_docs = recurSplitter.create_documents([txt_lin])

In [ ]:
with open('/content/space_shortened.csv') as spc:
  txt_spc = spc.read()

space_docs = recurSplitter.create_documents([txt_spc])

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings 
hfEmbed = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#The all-MiniLM has 384 vector elements. I suspect it may or may not work 
#with paLM. The alternate will be to work with instructor-xl. Uncomment 
#below code and execute

#hfEmbed = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl")

In [ ]:
from langchain.vectorstores import Chroma

space_chroma = Chroma.from_documents(documents=space_docs,
                                     embeddings=hfEmbed)

In [ ]:
lin_chroma = Chroma.from_documents(documents=linux_docs,
                                     embeddings=hfEmbed)

In [ ]:
# Starting the custom tool route

In [ ]:
lin_retriever =RetrievalQA.from_chain_type(llm=fact_llm, 
                                           chain_type="stuff", 
                                           retriever=lin_chroma.as_retriever())

In [ ]:
lin_retriever("What this documents are about?")

In [ ]:
space_retriever =RetrievalQA.from_chain_type(llm=fact_llm, 
                                           chain_type="stuff", 
                                           retriever=space_chroma.as_retriever())

In [ ]:
space_retriever("how many passengers are there?")

### The below code is extending the above retrivers as tools to the agents. Which is not required to execute

In [ ]:
tools = [
    Tool(
        name = "Space Farers",
        func=space_retriever.run,
        description="useful for when you need to answer questions about the space farers."
    ),
    Tool(
        name = "Linux Commands",
        func=lin_retriever.run,
        description="useful for when you need to answer questions about Linux commands."
    ),
]

In [ ]:
example_agent = initialize_agent(tools, 
                         fact_llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)

In [ ]:
example_agent.run("How many space farers are there?")

In [ ]:
# Starting the custom vectorstore agent route

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_router_agent,
    VectorStoreRouterToolkit,
    VectorStoreInfo,
)

In [ ]:
space_vectorstore_info = VectorStoreInfo(
    name="space",
    description="Used when need to answer about Space fares",
    vectorstore=space_chroma
)

In [ ]:
linux_vectorstore_info = VectorStoreInfo(
    name="linux",
    description="Used when need to answer about playbook",
    vectorstore=lin_chroma
)

In [ ]:
router_toolkit = VectorStoreRouterToolkit(
    vectorstores=[space_vectorstore_info,
                  linux_vectorstore_info],
    llm=fact_llm
)
agent_executor = create_vectorstore_router_agent(
    llm=fact_llm,
    toolkit=router_toolkit,
    verbose=True
)

In [ ]:
query = """How many Space fares are there?"""

In [ ]:
agent_executor.run(query)

In [ ]:
agent_executor.save_agent("multi_doc_agent.json")